# Gradient Boosting Regression

Another popular one is Gradient Boosting.  Similar to AdaBoost, Gradient Boosting works by adding sequential predictors.  However, instead of adding **weights**, this method tries to fit the new predictor to the **residual errors** made by the previous predictor.    

## Hypothesis function

The hypothesis function of gradient boosting is as follows:

$$
h(\mathbf{x}^{(i)}) = h_0(\mathbf{x}^{(i)}) + \alpha_1h_1(\mathbf{x}^{(i)}) + \cdots + \alpha_s h_s(\mathbf{x}^{(i)})
$$

Similar to AdaBoost, decision trees are typically used for each $h_s(\mathbf{x}^{(i)})$.

However, although they look similar, notice that 
1. No alpha is applied to the first predictor, because the learning is "sequential"
2. In addition, all alpha shares the same number.  Here, alpha is like the learning rate in regression.

You will clearly understand these differences as we go.
 
To simplify, we shall first talk about gradient boosting, assuming a regression problem.  We shall later move to classification which is trivial.

## Gradient Boosting for Regression

Firstly, let's look at the following equation where $h_0(\mathbf{x}^{(i)})$ is our first predictor and we would like to minimize the residual as follows:

$$h_0(\mathbf{x}^{(i)}) + \text{residual}_0 = y^{(i)} $$
$$ \text{residual}_0 =  y^{(i)} - h_0(\mathbf{x}^{(i)}) $$

That is, we would $y$ to be as close as $h$ such that residual is 0

$$ y^{(i)} = h_0(\mathbf{x}^{(i)}); \text{s.t.  residual} = 0$$

The question is that is it possible to add the second predictor $h_1(\mathbf{x}^{(i)})$ such that the residual is further reduced

$$ y^{(i)} = h_0(\mathbf{x}^{(i)}) + h_1(\mathbf{x}^{(i)}) $$

This equation can be written as:

$$h_1(\mathbf{x}^{(i)}) = y^{(i)} - h_0(\mathbf{x}^{(i)}) $$

This equation informs us that if we can find a subsequent predictor that can best fit the "residual" (i.e. $y^{(i)} - h_0(\mathbf{x}^{(i)})$), then we can improve the accuracy.

## Proof

Well, firstly, here is our loss function for regression:

$$J = \frac{1}{2}(y^{(i)} - h(\mathbf{x}^{(i)}))^2$$

In linear regression, we will find $\mathbf{w}$ that can minimize $J$ by finding the gradient of $J$ in respect to $\mathbf{w}$.

But in gradient boosting, we want to find $h$ that can minimize $J$.  Thus, we need to find the gradient of $J$ in respect to $h(\mathbf{x})$.   The equation can be written as follows:

$$\frac{\partial J}{\partial h(\mathbf{x}^{(i)})} = h(\mathbf{x}^{(i)}) - y^{(i)} $$

Based on gradient descent, we take the negative gradients for the update which is $y - h_0(x)$

$$h_1(\mathbf{x}^{(i)}) = - \frac{\partial J}{\partial h_0(\mathbf{x}^{(i)})} = -(h_0(\mathbf{x}^{(i)}) - y^{(i)}) = y^{(i)} - h_0(\mathbf{x}^{(i)})$$

or more generally (where $s$ is the index of predictor)

$$h_s(\mathbf{x}^{(i)}) = - \frac{\partial J}{\partial h_{s-1}(\mathbf{x}^{(i)})} = y^{(i)} - h_{s-1}(\mathbf{x}^{(i)})$$


## Classification?

In cross entropy, the loss function is

 $$J= y \lg h(\mathbf{x}) + (1 - y) \lg (1-h(\mathbf{x}))$$
 
The derivative of $J$ in respect to $h(\mathbf{x})$ will be:

$$\frac{\partial J}{\partial h_(\mathbf{x})} = h(\mathbf{x}) - y$$

This may look the same as mse, but note that our $h(\mathbf{x})$ (i.e., regression tree) outputs continuous values.  In order to transform $h(x)$ into discrete class, we shall transform using sigmoid function $g$ as follows:

$$g(h(\mathbf{x})) = g(\mathbf{z}) = \frac{1}{1+e^{-\mathbf{z}}}$$

For multiclass classification, $g$ is defined as the softmax function:

$$g(h(\mathbf{x})) = g(\mathbf{z}) = \frac{e^\mathbf{z}_c}{\displaystyle\sum_{i=1}^{k} e^\mathbf{z}_k}$$

Also remind that to use softmax function, we need to first one-hot encode our y.  And during prediction, we need to perform <code>np.argmax</code> along the `axis=1`.

## Adding learning rate

To make sure adding the subsequent predictor would not overfit our model, we shall add an learning rate $\alpha$ in front of this, which shall be the same across all predictors (different from AdaBoost where alpha is different across all predictors)

$$h_s(\mathbf{x}) = - \alpha \frac{\partial J}{\partial h_{s-1}(\mathbf{x})}$$

## How does it look for the third predictors?

$$ \text{residual}_1 =  y - (h_0(\mathbf{x}) + \alpha h_1(\mathbf{x}))$$

then we define $h_2(\mathbf{x})$ as 

$$h_2(\mathbf{x}) = \alpha(y - (h_0(\mathbf{x}) + \alpha h_1(\mathbf{x})))$$

And then repeat

The final prediction shall use the following hypothesis function:

$$
h(\mathbf{x}^{(i)}) = h_0(\mathbf{x}^{(i)}) + \alpha_1h_1(\mathbf{x}^{(i)}) + \cdots + \alpha_s h_s(\mathbf{x}^{(i)})
$$

Here, $\alpha$ is simply a fixed learning rate, same across all $h$.

## When do we stop?

1. When we reached desired iterations
2. When the residual does not decrease further using some validation set

## Summary of steps

1. Initialize the model as simply mean or some constant
2. Predict and calculate the residual
3. Let the next model fit the residual
4. Predict using the combined models and calculate the residual
5. Let the next model fit this residual
6. Simply repeat 4-5 until stopping criteria is reached

## 1. Scratch

In [1]:
from scipy.special import expit
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor

def grad(y, h):
    return y - h

def fit(X, y, models):
    
    models_trained = []
    
    #using DummyRegressor is a good technique for starting model
    first_model = DummyRegressor(strategy='mean')
    first_model.fit(X, y)
    models_trained.append(first_model)
    
    #fit the estimators
    for i, model in enumerate(models):
        #predict using all the weak learners we trained up to
        #this point
        y_pred = predict(X, models_trained)
        
        #errors will be the total errors maded by models_trained
        residual = grad(y, y_pred)
        
        #fit the next model with residual
        model.fit(X, residual)
        
        models_trained.append(model)
        
    return models_trained
        
def predict(X, models):
    learning_rate = 0.1  ##hard code for now
    f0 = models[0].predict(X)  #first use the dummy model
    boosting = sum(learning_rate * model.predict(X) for model in models[1:])
    return f0 + boosting

## Let's use our scratch code!

In [2]:
# Regression

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

X, y = load_diabetes(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

n_estimators = 200
tree_params = {'max_depth': 1}
models = [DecisionTreeRegressor(**tree_params) for _ in range(n_estimators)]

#fit the models
models = fit(X_train, y_train, models)

#predict
y_pred = predict(X_test, models)

#print metrics
print("Our MSE: ", mean_squared_error(y_test, y_pred))

Our MSE:  2714.188989170066


## 2. Sklearn 

sklearn has implemented GradientBoosting under the API of <code>GradientBoostingClassifier</code> for classification and <code>GradientBoostingRegressor</code> for regression.

In [3]:
#Compare to sklearn: ls is the same as our mse
sklearn_model = GradientBoostingRegressor(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1,
)

y_pred_sk = sklearn_model.fit(X_train, y_train).predict(X_test)

#print metrics
print("Sklearn MSE: ", mean_squared_error(y_test, y_pred_sk))

Sklearn MSE:  2715.7388118438184


## 3. XGBoost

XGBoost is an optimized distributed gradient boosting, designed to be more efficient, flexible, and portable (Chen and Guestrin 2016).  In fact, XGBoost is often an important component of the winning entries in ML competitions (e.g., Kaggle).  XGBoost also offers several nice features, such as automatically taking care of early stopping: XGBoost’s API is quite similar to Scikit-Learn’s:

In [4]:
#make sure to pip install xgboost
#for mac guys, do "brew install libomp" which installs openMP library
#required for XGBoost

import xgboost

xgb_reg = xgboost.XGBRegressor(early_stopping_rounds=2) 

#not improved after 2 iterations
xgb_reg.fit(X_train, y_train,
                eval_set=[(X_test, y_test)])
y_pred = xgb_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)  #notice we are using mse while xgb uses root mse

[0]	validation_0-rmse:64.95543
[1]	validation_0-rmse:59.66044
[2]	validation_0-rmse:57.59860
[3]	validation_0-rmse:56.58414
[4]	validation_0-rmse:56.67145
[5]	validation_0-rmse:56.75270
MSE: 3201.7650905241712


### When to use Boosting

Let's summarize some useful info about Gradient Boosting:

Advantages:
1. Extremely powerful - especially useful for heterogeneous data (e.g., house price, number of bedrooms). 

Disadvantages:
1. They cannot be parallelized.  Obvious since they are sequential predictors.
2. They can easily overfit, thus require careful choice of estimators or the use of regularization such as max_depth.
3. When we talk about homogeneous data such as images, videos, audio, text, or huge amount of data, deep learning works better.

## Workshop

1. What are some observable differences between AdaBoost and Gradient Boosting?
2. What is the main idea of Gradient Boosting?
3. Why do we find the gradient of $J$ in respect to $h$?
4. Why Chaky say that ``finding the next $h$ is simply creating a model to fit the residuals"?
5. For Gradient Boosting, we got some $\alpha$ to set?  What is the range of this value?  Is it same across all models?
6. Why do we need learning rate?
7. In Gradient Boosting, why the first model does not have alpha?
8. In Gradient Boosting, when do we stop?
9. In the code, it was written `DummyRegressor(strategy='mean')`.  What is it?    For classification, what will be a good first model?